In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/sec6/
%ls -a
!pip install pytorch-gradcam

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

import os
import scipy
import random
from tqdm import tqdm
import glob

from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve

import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.optim as optim

import torchvision
from torchvision import models
from torchvision import datasets
from torchvision import transforms as transforms

from gradcam.utils import visualize_cam
from gradcam import GradCAM, GradCAMpp


In [ ]:
path = "/content/drive/MyDrive/sec6"
im_fd = "/cell_images/"

In [ ]:
folder = ['Uninfected','Parasitized']
file0 = glob.glob(path + im_fd + folder[0] +"/*.png")[0]
print(file0)
image = Image.open(file0)
plt.imshow(image)
np.array(image)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")
image_size = 128;

In [ ]:
SEED = 42

def seed_fix(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.use_deterministic_algorithms = True
    torch.backends.cudnn.deterministic = True

    worker_seed = torch.initial_seed() % 2**32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

seed_fix(SEED)
generator = torch.Generator()
generator.manual_seed(SEED)

In [ ]:
import glob
import random
import numpy as np
from PIL import Image
from tqdm import tqdm


folder = ['Uninfected', 'Parasitized']
X, y = [], []
usenumber = 1000


for i, folname in enumerate(folder):
    files = glob.glob(f"{path + im_fd + folname}/*.png")
    sampled_files = random.sample(files, usenumber)


    for file in tqdm(sampled_files, desc=f"Processing {folname}"):
        image = Image.open(file).convert("RGB").resize((image_size, image_size))
        data = np.asarray(image)
        X.append(data)
        y.append(i)


In [ ]:

X = np.array(X).astype(np.float32).transpose(0,3,1,2)/255
y = np.array(y)
Nall = X.shape[0]
tensor_X = torch.tensor(X, dtype=torch.float32)
tensor_y = torch.tensor(y, dtype=torch.int64)
dataset = torch.utils.data.TensorDataset(tensor_X,tensor_y)
n_train = int(Nall * 0.7)
n_val = int(Nall * 0.2)
n_test = Nall - n_train - n_val
train_x, val_x, test_x = torch.utils.data.random_split(dataset, [n_train, n_val,n_test])
print("train =",n_train,",validation =",n_val,",Test =",n_test)
batch_size = 64
train_dataloader = DataLoader(train_x, batch_size=batch_size, shuffle=True)
val_dataloader   = DataLoader(val_x,   batch_size=batch_size, shuffle=False)
test_dataloader  = DataLoader(test_x,  batch_size=1, shuffle=False)


In [ ]:
Nall = X.shape[0]

In [ ]:
class CNN(nn.Module):
    def __init__(self, input_shape=(3,128,128),output_size=2):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(in_channels=input_shape[0], out_channels=16, kernel_size=3, padding='same'),nn.ReLU(),nn.MaxPool2d(2,2))
        self.conv2 = nn.Sequential(nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, padding='same'),nn.ReLU(),nn.MaxPool2d(2,2))
        self.conv3 = nn.Sequential(nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding='same'),nn.ReLU(),nn.MaxPool2d(2,2))
        self.conv4 = nn.Sequential(nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding='same'),nn.ReLU(),nn.MaxPool2d(2,2))
        self.flatten = nn.Flatten()
        self.dropout = nn.Dropout(0.3)
        self.CNN_outshape = self._get_conv_output(input_shape)
        self.linear = nn.Linear(self.CNN_outshape, output_size)
    def _get_conv_output(self, shape):
        bs = 1
        dummy_x = torch.empty(bs, *shape)
        x = self._forward_features(dummy_x)
        CNN_outshape = x.flatten(1).size(1)
        return CNN_outshape
    def _forward_features(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.dropout(x)
        x = self.flatten(x)
        return x
    def forward(self, x):
        x = self._forward_features(x)
        x = self.linear(x.flatten(1))
        return x
model = CNN().to(device)
print(model)

In [ ]:
transform = transforms.Compose([
    transforms.RandomResizedCrop((image_size,image_size)),
    transforms.RandomHorizontalFlip(p=0.4),
    transforms.RandomVerticalFlip(p=0.4),
    transforms.RandomRotation(degrees=[-7.5, 7.5])
    ]
)

In [ ]:

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay= 0.005)

def train(train_loader): #Training
    model.train()
    running_loss = 0
    correct = 0
    total = len(train_loader.dataset)
    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        #data augumentation
        #images = transform(images)
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        predicted = outputs.max(1, keepdim=True)[1]
        labels = labels.view_as(predicted)
        correct += predicted.eq(labels).sum().item()
    train_loss = running_loss / len(train_loader)
    train_acc = correct / total
    return train_loss, train_acc

def valid(test_loader): #Validation
    model.eval()
    running_loss = 0
    correct = 0
    total = len(test_loader.dataset)
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            predicted = outputs.max(1, keepdim=True)[1]
            labels = labels.view_as(predicted)
            correct += predicted.eq(labels).sum().item()
    val_loss = running_loss / len(test_loader)
    val_acc = correct / total
    return val_loss, val_acc

#空配列
acc_list = []
loss_list = []
val_loss_list = []
val_acc_list = []

In [ ]:
nepoch = 300

for epoch in range(nepoch):
    loss, acc = train(train_dataloader)
    val_loss, val_acc = valid(val_dataloader)
    print('epoch %d, loss: %.4f acc: %.4f val_loss: %.4f val_acc: %.4f' % (epoch, loss,acc, val_loss, val_acc))
    loss_list.append(loss)
    acc_list.append(acc)
    val_loss_list.append(val_loss)
    val_acc_list.append(val_acc)

In [ ]:

torch.save(model, 'model.pt')
model = torch.load('model.pt')

In [ ]:

print(f'正解率：{val_acc_list[-1] * 100:.2f} %')


fig, ax = plt.subplots(2, 1, figsize=(8, 10))


ax[0].plot(range(nepoch), loss_list, color='red', linestyle='-', label='train_loss')
ax[0].plot(range(nepoch), val_loss_list, color='blue', linestyle='-', label='val_loss')
ax[0].legend()
ax[0].set_xlabel('エポック')
ax[0].set_ylabel('損失')
ax[0].set_title('学習と検証の損失曲線')


ax[1].plot(range(nepoch), acc_list, color='blue', linestyle='-', label='acc')
ax[1].plot(range(nepoch), val_acc_list, color='green', linestyle='-', label='val_acc')
ax[1].legend()
ax[1].set_xlabel('エポック')
ax[1].set_ylabel('精度')
ax[1].set_title('学習と検証の精度曲線')


plt.tight_layout()
plt.show()


In [ ]:

def test_alternative(test_loader):
    prob, pred, true = [], [], []
    model.eval()
    running_loss, correct = 0, 0
    total = len(test_loader.dataset)

    with torch.no_grad():
        for batch in test_loader:

            images, labels = map(lambda x: x.to(device), batch)


            outputs = model(images)
            lprob, predicted = torch.max(outputs, dim=1, keepdim=True)

            labels = labels.view_as(predicted)
            correct += (predicted == labels).sum().item()

            prob.extend(scipy.special.expit(outputs[:, 1].cpu().numpy()))
            pred.extend(predicted.cpu().numpy())
            true.extend(labels.cpu().numpy())

    return prob, pred, true


prob, pred, true = test_alternative(test_dataloader)


In [ ]:

plt.figure(figsize = (25, 25))
for i in range(20):
    plt.subplot(5, 4, i + 1)
    plt.axis("off")
    if pred[i] == true[i]:
        plt.title("pred:"+str(pred[i].astype(np.uint8))+' - '+"true:"+str(true[i].astype(np.uint8))+'\n'+'Prob.(Y=1) = %.4f' % (prob[i]))
    else:
        plt.title("pred:"+str(pred[i].astype(np.uint8))+' - '+"true:"+str(true[i].astype(np.uint8))+'\n'+'Prob.(Y=1) = %.4f' % (prob[i]), color = "red") # 分類が間違っていた場合，赤字で書き込む

    tmp = test_x[i][0].to('cpu').detach().numpy().copy()
    tmp = tmp.transpose(1, 2, 0)
    img_pil = Image.fromarray((tmp*255).astype(np.uint8))
    plt.imshow(img_pil)




In [ ]:

cmat = confusion_matrix(true, pred)
print(cmat)

tn, fp, fn, tp = cmat.flatten()

acc = round((tp+tn)/(tp+tn+fp+fn),4)
sen = round(tp/(tp+fn),4)
spe = round(tn/(tn+fp),4)
ppv = round(tp/(tp+fp),4)
npv = round(tn/(tn+fn),4)
print("acc=",acc,"sen=",sen," ,spe=",spe," ,ppv=",ppv," ,npv=",npv)

